Inspired by https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

In [2]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path+"/scripts")

In [3]:
import pandas as pd
import pickle

with open("../data/all_reviews.pkl","rb") as f:
    reviews = pickle.load(f)
    

In [40]:
# with open("../data/job_titles.txt","r") as f:
#     common_tech_titles = [ l.replace("\n","").strip() for l in f ]
import string
import re
from DataPrep import cleanTitle


common_tech_titles = pd.read_csv("../data/job_titles.txt")

common_tech_titles["Job_Title"] = common_tech_titles.Job_Title.apply(cleanTitle)

display(common_tech_titles.Job_Title.value_counts())


software engineer                          3
network architect                          2
support specialist                         2
system architect                           2
systems software engineer                  2
                                          ..
data center support specialist             1
computer programmer                        1
desktop support manager                    1
computer information research scientist    1
customer support administrator             1
Name: Job_Title, Length: 70, dtype: int64

## N-Gram with Top N Cosine

In [41]:
# clean_job_titles = reviews.clean_job_title.tolist()
clean_job_titles_filtered = reviews.clean_job_title.value_counts()
clean_job_titles_filtered = clean_job_titles_filtered.loc[clean_job_titles_filtered > 1].index.tolist()

clean_job_titles_filtered.extend(set(common_tech_titles.Job_Title.tolist()))

print(len(clean_job_titles_filtered))

106249


In [6]:
import re

def ngrams(string, n=3):
    #string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in {}:'.format(clean_job_titles_filtered[0]))
print( ngrams(clean_job_titles_filtered[0]) )

All 3-grams in sales associate:
['sal', 'ale', 'les', 'es ', 's a', ' as', 'ass', 'sso', 'soc', 'oci', 'cia', 'iat', 'ate']


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(clean_job_titles_filtered)

print(len(vectorizer.get_feature_names()))

9125


In [8]:
print(clean_job_titles_filtered[0])
print( tf_idf_matrix[0] )

sales associate
  (0, 1202)	0.26292540368052825
  (0, 3570)	0.28059718134382955
  (0, 1708)	0.21463563116965367
  (0, 5433)	0.28027856751841695
  (0, 6842)	0.28019328630429796
  (0, 6915)	0.2760705507206696
  (0, 1192)	0.2377285822615709
  (0, 60)	0.2563267698347386
  (0, 6606)	0.3282008167188559
  (0, 2629)	0.2861858988943331
  (0, 4374)	0.2964736880627908
  (0, 1044)	0.29332914743884886
  (0, 6652)	0.29497902408255716


In [9]:
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [10]:
# import time
# t1 = time.time()
# matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.7)
# t = time.time()-t1
# print("SELFTIMED:", t)

In [11]:
from sparse_dot_topn import awesome_cossim_topn
import time

t1 = time.time()

matches = awesome_cossim_topn(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.7, use_threads=True, n_jobs=11)

t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 35.447951793670654


In [12]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [13]:
matches_df = get_matches_df(matches, clean_job_titles_filtered, top=False)
# matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)

,left_side,right_side,similairity
342479,head teller assistant branch manager,teller branch manager,0.740180
28020,solution consultant,sap solution consultant,0.810474
89358,verizon mobile expert,verizon expert,0.723070
399050,financial leadership program intern,financial leadership program intern,1.000000
168700,lab medical assistant,medical assistant,0.755844
332572,marketing sales assistant,assistant marketing,0.770405
561019,cashier garden associate tool rental associate,cashier garden associate,0.754303
585686,asesora ventas cajera,asesora ventas,0.768300
426910,forklift operator material handler order puller,order puller forklift operator,0.735619
124352,commercial sales assistant,commercial assistant,0.825269


In [14]:
matches_df.sort_values(['similairity'], ascending=True).head(10)

,left_side,right_side,similairity
48399,social media manager,social media team,0.700001
337806,social media team,social media manager,0.700001
604890,design supervisor project manager,design engineer project manager,0.700001
281381,técnico telecomunicações pleno,técnico eletrônica telecomunicações,0.700001
439354,técnico eletrônica telecomunicações,técnico telecomunicações pleno,0.700001
398185,garden associate receiving,sales associate receiving associate,0.700002
502926,salesman merchandising specialist,merchandising sales,0.700002
320966,rental technician,fleet rental technician,0.700002
517101,fleet rental technician,rental technician,0.700002
414301,net developer intern,developer internship,0.700003


In [42]:
filtered_jobs = matches_df.loc[matches_df.left_side.isin(set(common_tech_titles.Job_Title.tolist())),["left_side","right_side"]]
print(filtered_jobs.shape)

pd.set_option("display.max_colwidth",500)
display(filtered_jobs.groupby("left_side")["right_side"].apply(list).reset_index())
pd.reset_option("display.max_colwidth")


(1316, 2)


,left_side,right_side
0,application developer,"[application developer, application developer, application developer sap, application developer team, application developer associate, application developer analyst, application developer specialist, net application developer, rational application developer, application developer intern, application developer, application developer, application developer sap, application developer team, application developer associate, application developer analyst, application developer specialist, net appl..."
1,application support analyst,"[application support analyst, application support analyst, applications support analyst, technical application support analyst, clinical application support analyst, application support, business application support analyst, application production support analyst, production application support analyst, application technical support analyst, application support analyst, application support analyst, applications support analyst, technical application support analyst, clinical application supp..."
2,applications engineer,"[applications engineer, applications engineer, applications engineer, applications engineer sales engineering, applications engineer intern, application operations engineer, application sales engineer, application engineer, technical applications engineer, applications engineering intern, applications engineer, applications engineer, applications engineer, applications engineer sales engineering, applications engineer intern, application operations engineer, application sales engineer, appli..."
3,associate developer,"[associate developer, associate developer, developer associate, associate development, developer developer, developer, software associate developer, associate development manager, store developer, associate software developer, associate developer, associate developer, developer associate, associate development, developer developer, developer, software associate developer, associate development manager, store developer, associate software developer]"
4,chief information officer,"[chief information officer, chief information officer, chief information office, chief information officer cio, chief information security officer, information officer, chief information systems, chief information officer, chief information officer, chief information office, chief information officer cio, chief information security officer, information officer, chief information systems]"
...,...,...
65,technology systems administrator,"[technology systems administrator, technology systems administrator, technology system administrator, systems administrator technology consultant, technology specialist system administrator, systems administrator, systems administrator, technology administrator, tier systems administrator, system administrator technology, technology systems administrator, technology systems administrator, technology system administrator, systems administrator technology consultant, technology specialist syst..."
66,telecommunications specialist,"[telecommunications specialist, telecommunications specialist, telecommunication specialist, telecommunications operations specialist, communications specialist, telecommunications, telecommunications sales, technology telecommunications specialist, telecommunications specialist project manager, telecommunications manager, telecommunications specialist, telecommunications specialist, telecommunication specialist, telecommunications operations specialist, communications specialist, telecommun..."
67,web administrator,"[web administrator, web administrator, web administrator, lab administrator, web content administrator, web administrator, web administrator, web administrator, lab administrator, web content administrator, web administrator, web administrator, web administrator, lab administrator, web content administrator]"
68,web developer,"[web developer, web developer, web developer, web developer 

In [43]:
sum(reviews.clean_job_title.isin(filtered_jobs.right_side.tolist()))

42590

## TF-IDF with K-Means

In [17]:

clean_job_titles = reviews.clean_job_title.unique().tolist()
print(len(clean_job_titles))

506654


In [18]:
import re

def ngrams(string, n=3):
    #string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in {}:'.format(common_tech_titles[0]))
print( ngrams(common_tech_titles[0]) )

All 3-grams in cloud architect:
['clo', 'lou', 'oud', 'ud ', 'd a', ' ar', 'arc', 'rch', 'chi', 'hit', 'ite', 'tec', 'ect']


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(set(common_tech_titles.Job_Title.tolist()))

print(len(vectorizer.get_feature_names()))

340


In [20]:
from sklearn.neighbors import NearestNeighbors
import time

nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tf_idf_matrix)

In [21]:
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(clean_job_titles)
t = time.time()-t1
print("COMPLETED IN:", t)

getting nearest n...
COMPLETED IN: 7.2571940422058105


In [22]:
print('finding matches...')
kmeans_matches = []
for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), common_tech_titles[j[0]],clean_job_titles[i]]
    kmeans_matches.append(temp)
print('Building data frame...')  
kmeans_matches = pd.DataFrame(kmeans_matches, columns=['Match confidence (lower is better)','left_side','right_side'])
print('Done')

finding matches...
Building data frame...
Done


In [23]:
idx = kmeans_matches["Match confidence (lower is better)"] < .4
kmeans_filtered_jobs = kmeans_matches.loc[idx,["left_side","right_side"]]
print(sum(idx))

pd.set_option("display.max_colwidth",700)
display( kmeans_filtered_jobs.groupby("left_side").right_side.apply(list).reset_index() )
pd.reset_option("display.max_colwidth")

1442


,left_side,right_side
0,application developer,"[application developer, sap abap application developer, mobile application developer, oracle application developer, application developer latam, application develope, application developer head, android application developer, ios application developer, mean stack application developer, application developer sql, android mobile application developer, ios mobile application developer, application developer mobile ios, websphere application developer, application developer j2ee, tririga application developer, application developer sap, application developer ios, application developer qlikview cognos, application developer oracle apps, maximo application developer, application developer lotu..."
1,application support analyst,"[application support analyst, business application support analyst, hcm application support analyst, peachtree accounting application support analyst, unix application support analyst, mainframe application support analyst, application support analyst oracle apps, application support analyst team, application support business analyst, level application support analyst, application support analysis qnxt, tax support analyst cash applications, linux application support analyst, business analyst application support, analyst application support, application support analyst offshore, level banking application support analyst, system application support analyst, application monitoring support ..."
2,applications engineer,"[applications engineer, delcam applications engineer, staff field applications engineer, field applications engineer, applications engineer hvcb, applications engineer cad cam, applications tailoring engineer, foundry applications engineer, applications engineer devops, applications engineer ic5, principle applications engineer, lighting applications engineer, oracle applications engineer, sas applications engineer, applications engineer cvp, sales applications engineer]"
3,associate developer,"[associate developer, associate developer dot net, associate oracle sql developer, associate etl developer, associate sql etl developer, associate sql developer, associate marklogic developer, associate full stack developer, associate developer full time]"
4,chief information officer,"[chief information officer, interim chief information officer, chief information office, chief information officer cio, chief information office cio, chief information officer west region, regional chief information officer]"
...,...,...
57,technology systems administrator,"[technology systems administrator, technology systemadministrator, nexgen technology systems administrator]"
58,telecommunications specialist,"[telecommunications specialist, engineering telecommunications specialist, telecommunications repair specialist, telecommunications account specialist, military telecommunications specialist, analyst telecommunications specialist, telecommunications systems specialist, telecommunications specialist csr]"
59,web administrator,"[web administrator, marketing web administrator, training advisor•web administrator, web hosting administrator]"
60,web developer,"[web developer, full stack web developer, web developer web developer, php web developer, ctl web developer, web developer oracle, web developer html5 developer, web developer jefe equipo, drupal web developer, angular web developer, mobile web developer, web developer fullstack, role web developer, global web developer, web developer sme, oracle sql web developer, j2ee web developer, staff web developer ebt, eftps web developer, sfdc web developer, web developer developer]"


In [24]:
sum(reviews.clean_job_title.isin(kmeans_filtered_jobs.right_side.tolist()))

36276

Merge both datsets

In [52]:

finalized_job_list = pd.concat([filtered_jobs, kmeans_filtered_jobs]).drop_duplicates()

print(finalized_job_list.shape)

finalized_job_list = finalized_job_list.merge(common_tech_titles, how="left", left_on="left_side",right_on="Job_Title")\
                        .rename(columns={"right_side":"job_title","Job_Title":"common_job_title"})

del finalized_job_list["left_side"]

display( finalized_job_list.head() )

finalized_job_list.to_csv("../data/finalized_job_list.csv", index=False)

(1855, 2)


,job_title,common_job_title,Job_Family
0,software engineer,software engineer,Software/Application Developer
1,software engineer,software engineer,Software/Application Developer
2,software engineer,software engineer,Software/Application Developer
3,software engineer software engineer,software engineer,Software/Application Developer
4,software engineer software engineer,software engineer,Software/Application Developer


In [26]:
sum(reviews.clean_job_title.isin(finalized_job_list.right_side.tolist()))

45701

Remove Jobs...

In [28]:
filter_titles = finalized_job_list.right_side.tolist()

exclude_keywords = [
    "camp"
    ,"tutor"
    ,"professor"
    ,"entry"
    ,"entries"
    ,"teacher"
    ,"audio"
    ,"civil"
    ,"facility"
    ,"facilities"
    ,"industrial"
    ,"metals"  
    ,"lab"
]

def excludeJobTItles(job):
    tokens = str(job).split(" ")
    return len( [ word for word in tokens if word in exclude_keywords ] ) == 0

filter_titles_2 = list( filter(excludeJobTItles, filter_titles) )

In [29]:
sum(reviews.clean_job_title.isin(filter_titles_2))

45687

In [30]:
import pandas as pd

pd.DataFrame(filter_titles_2,columns=["clean_job_title"]).to_csv("../data/filter_job_titles.csv",index=False)

## Group Job Titles

In [32]:
import pandas as pd

# filter_job_titles = pd.read_csv("../data/filter_job_titles.csv")



,left_side,right_side
120,software engineer,software engineer
124,software engineer,software engineer software engineer
125,software engineer,software engineering
126,software engineer,software engineer engineer
127,software engineer,engineer software engineer
...,...,...
499417,database administrator,database administrator legal affairs
501017,network systems administrator,bhqn network systems administrator
504660,programmer analyst,programmer analyst dba
505423,systems analyst,scheduling systems analyst
